In [ ]:
import sys
isColab = "google.colab" in sys.modules
# this also works: 
# isColab = "COLAB_GPU" in os.environ

if isColab:
    from google.colab import drive 
    drive.mount("/content/drive", force_remount=True) 
    
    student_colab_path = ("/content/drive/MyDrive"
        + "/detecting-drinking-episodes")
    sys.path.append(student_colab_path)

Mounted at /content/drive


In [ ]:
import pickle
with open("/content/drive/MyDrive/detecting-drinking-episodes/data/DC6359_clean_tac_data", 'rb') as f:
  data = pickle.load(f)
dataframe = data['data']
# test_dataframe = dataframe[dataframe['pid'] == "BK7610"]
test_dataframe = dataframe[dataframe['pid'] == 'DC6359']
#dataframe = dataframe[(dataframe['pid'] != 'BK7610') & (dataframe['pid']!='BU4707')]


dataframe['pid'].unique() 
dataframe.head()

,pid,x,y,z,time,tac
88317,DC6359,0.0045,-0.0085,-0.0069,1493736440,0
88314,DC6359,0.0003,-0.0001,0.0038,1493736440,0
88317,DC6359,0.0045,-0.0085,-0.0069,1493736440,0
88314,DC6359,0.0003,-0.0001,0.0038,1493736440,0
88317,DC6359,0.0045,-0.0085,-0.0069,1493736440,0


In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import scipy
from scipy.stats import skew, kurtosis
from scipy import stats
import tensorflow as tf
import math
from math import ceil

In [ ]:
train_df = dataframe.drop('pid', axis=1)
train_df.head()

,x,y,z,time,tac
88317,0.0045,-0.0085,-0.0069,1493736440,0
88314,0.0003,-0.0001,0.0038,1493736440,0
88317,0.0045,-0.0085,-0.0069,1493736440,0
88314,0.0003,-0.0001,0.0038,1493736440,0
88317,0.0045,-0.0085,-0.0069,1493736440,0


In [ ]:
df_train = train_df
df_train.head() 

,x,y,z,time,tac
88317,0.0045,-0.0085,-0.0069,1493736440,0
88314,0.0003,-0.0001,0.0038,1493736440,0
88317,0.0045,-0.0085,-0.0069,1493736440,0
88314,0.0003,-0.0001,0.0038,1493736440,0
88317,0.0045,-0.0085,-0.0069,1493736440,0


In [ ]:
x_list = []
y_list = []
z_list = []
train_labels = []

window_size = 100
step_size = 50

# creating overlaping windows of size window-size 100
for i in range(0, df_train.shape[0] - window_size, step_size):
    xs = df_train['x'].values[i: i + 100]
    ys = df_train['y'].values[i: i + 100]
    zs = df_train['z'].values[i: i + 100]
    label = stats.mode(df_train['tac'][i: i + 100])[0][0]

    x_list.append(xs)
    y_list.append(ys)
    z_list.append(zs)
    train_labels.append(label)

# Statistical Features on raw x, y and z in time domain 
X_train = pd.DataFrame()

# mean
X_train['tac'] = pd.Series(train_labels).apply(lambda x: x)
############### 
X_train['x_mean'] = pd.Series(x_list).apply(lambda x: x.mean())
X_train['y_mean'] = pd.Series(y_list).apply(lambda x: x.mean())
X_train['z_mean'] = pd.Series(z_list).apply(lambda x: x.mean())

# std dev
X_train['x_std'] = pd.Series(x_list).apply(lambda x: x.std())
X_train['y_std'] = pd.Series(y_list).apply(lambda x: x.std())
X_train['z_std'] = pd.Series(z_list).apply(lambda x: x.std())

# avg absolute diff
X_train['x_aad'] = pd.Series(x_list).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['y_aad'] = pd.Series(y_list).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['z_aad'] = pd.Series(z_list).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))

# min
X_train['x_min'] = pd.Series(x_list).apply(lambda x: x.min())
X_train['y_min'] = pd.Series(y_list).apply(lambda x: x.min())
X_train['z_min'] = pd.Series(z_list).apply(lambda x: x.min())

# max
X_train['x_max'] = pd.Series(x_list).apply(lambda x: x.max())
X_train['y_max'] = pd.Series(y_list).apply(lambda x: x.max())
X_train['z_max'] = pd.Series(z_list).apply(lambda x: x.max())

# max-min diff
X_train['x_maxmin_diff'] = X_train['x_max'] - X_train['x_min']
X_train['y_maxmin_diff'] = X_train['y_max'] - X_train['y_min'] 
X_train['z_maxmin_diff'] = X_train['z_max'] - X_train['z_min']

# median
X_train['x_median'] = pd.Series(x_list).apply(lambda x: np.median(x))
X_train['y_median'] = pd.Series(y_list).apply(lambda x: np.median(x))
X_train['z_median'] = pd.Series(z_list).apply(lambda x: np.median(x))

# median abs dev 
X_train['x_mad'] = pd.Series(x_list).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['y_mad'] = pd.Series(y_list).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['z_mad'] = pd.Series(z_list).apply(lambda x: np.median(np.absolute(x - np.median(x))))

# interquartile range
X_train['x_IQR'] = pd.Series(x_list).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['y_IQR'] = pd.Series(y_list).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['z_IQR'] = pd.Series(z_list).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))

# negtive count
X_train['x_neg_count'] = pd.Series(x_list).apply(lambda x: np.sum(x < 0))
X_train['y_neg_count'] = pd.Series(y_list).apply(lambda x: np.sum(x < 0))
X_train['z_neg_count'] = pd.Series(z_list).apply(lambda x: np.sum(x < 0))

# positive count
X_train['x_pos_count'] = pd.Series(x_list).apply(lambda x: np.sum(x > 0))
X_train['y_pos_count'] = pd.Series(y_list).apply(lambda x: np.sum(x > 0))
X_train['z_pos_count'] = pd.Series(z_list).apply(lambda x: np.sum(x > 0))

# values above mean
X_train['x_above_mean'] = pd.Series(x_list).apply(lambda x: np.sum(x > x.mean()))
X_train['y_above_mean'] = pd.Series(y_list).apply(lambda x: np.sum(x > x.mean()))
X_train['z_above_mean'] = pd.Series(z_list).apply(lambda x: np.sum(x > x.mean()))

# skewness
X_train['x_skewness'] = pd.Series(x_list).apply(lambda x: stats.skew(x))
X_train['y_skewness'] = pd.Series(y_list).apply(lambda x: stats.skew(x))
X_train['z_skewness'] = pd.Series(z_list).apply(lambda x: stats.skew(x))

# kurtosis
X_train['x_kurtosis'] = pd.Series(x_list).apply(lambda x: stats.kurtosis(x))
X_train['y_kurtosis'] = pd.Series(y_list).apply(lambda x: stats.kurtosis(x))
X_train['z_kurtosis'] = pd.Series(z_list).apply(lambda x: stats.kurtosis(x))

# energy
X_train['x_energy'] = pd.Series(x_list).apply(lambda x: np.sum(x**2)/100)
X_train['y_energy'] = pd.Series(y_list).apply(lambda x: np.sum(x**2)/100)
X_train['z_energy'] = pd.Series(z_list).apply(lambda x: np.sum(x**2/100))

# avg resultant
X_train['avg_result_accl'] = [i.mean() for i in ((pd.Series(x_list)**2 + pd.Series(y_list)**2 + pd.Series(z_list)**2)**0.5)]

# signal magnitude area
X_train['sma'] =    pd.Series(x_list).apply(lambda x: np.sum(abs(x)/100)) + pd.Series(y_list).apply(lambda x: np.sum(abs(x)/100)) \
                  + pd.Series(z_list).apply(lambda x: np.sum(abs(x)/100))

<ipython-input-7-163605c42be6>:14: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = stats.mode(df_train['tac'][i: i + 100])[0][0]


In [ ]:
X_train.head()

,tac,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,...,y_skewness,z_skewness,x_kurtosis,y_kurtosis,z_kurtosis,x_energy,y_energy,z_energy,avg_result_accl,sma
0,0,-0.000522,-0.001690,-0.001607,0.012240,0.007164,0.010232,0.008288,0.005044,0.007790,...,0.378103,-0.236652,4.863522,2.100309,1.599092,0.000150,0.000054,0.000107,0.014540,0.021559
1,0,-0.004428,0.000969,-0.000996,0.009895,0.008558,0.012174,0.007525,0.006547,0.008964,...,0.127128,0.081856,0.596410,0.675141,0.725077,0.000118,0.000074,0.000149,0.016146,0.024185
2,0,-0.002430,0.003917,0.000383,0.018988,0.013274,0.018427,0.013756,0.009328,0.013305,...,1.514645,0.062866,0.578144,6.637568,1.676540,0.000366,0.000192,0.000340,0.024918,0.036956
3,0,-0.003570,0.003626,0.000294,0.026215,0.014554,0.020006,0.020313,0.010637,0.013874,...,1.320550,-0.164258,0.320117,4.299226,1.930405,0.000700,0.000225,0.000400,0.030917,0.045112
4,0,-0.003775,0.003096,-0.000126,0.023270,0.012352,0.014846,0.018516,0.010172,0.010447,...,0.455842,-0.522619,1.158065,-0.333681,3.575382,0.000556,0.000162,0.000220,0.026454,0.039169


In [ ]:
x_list_fft = pd.Series(x_list).apply(lambda x: np.abs(np.fft.fft(x))[1:51])
y_list_fft = pd.Series(y_list).apply(lambda x: np.abs(np.fft.fft(x))[1:51])
z_list_fft = pd.Series(z_list).apply(lambda x: np.abs(np.fft.fft(x))[1:51])

# Statistical Features on raw x, y and z in frequency domain
# FFT mean
X_train['x_mean_fft'] = pd.Series(x_list_fft).apply(lambda x: x.mean())
X_train['y_mean_fft'] = pd.Series(y_list_fft).apply(lambda x: x.mean()) 
X_train['z_mean_fft'] = pd.Series(z_list_fft).apply(lambda x: x.mean())

# FFT std dev
X_train['x_std_fft'] = pd.Series(x_list_fft).apply(lambda x: x.std())
X_train['y_std_fft'] = pd.Series(y_list_fft).apply(lambda x: x.std())
X_train['z_std_fft'] = pd.Series(z_list_fft).apply(lambda x: x.std())

# FFT avg absolute diff
X_train['x_aad_fft'] = pd.Series(x_list_fft).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['y_aad_fft'] = pd.Series(y_list_fft).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['z_aad_fft'] = pd.Series(z_list_fft).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))

# FFT min
X_train['x_min_fft'] = pd.Series(x_list_fft).apply(lambda x: x.min())
X_train['y_min_fft'] = pd.Series(y_list_fft).apply(lambda x: x.min())
X_train['z_min_fft'] = pd.Series(z_list_fft).apply(lambda x: x.min())

# FFT max
X_train['x_max_fft'] = pd.Series(x_list_fft).apply(lambda x: x.max())
X_train['y_max_fft'] = pd.Series(y_list_fft).apply(lambda x: x.max())
X_train['z_max_fft'] = pd.Series(z_list_fft).apply(lambda x: x.max())

# FFT max-min diff
X_train['x_maxmin_diff_fft'] = X_train['x_max_fft'] - X_train['x_min_fft']
X_train['y_maxmin_diff_fft'] = X_train['y_max_fft'] - X_train['y_min_fft']
X_train['z_maxmin_diff_fft'] = X_train['z_max_fft'] - X_train['z_min_fft']

# FFT median
X_train['x_median_fft'] = pd.Series(x_list_fft).apply(lambda x: np.median(x))
X_train['y_median_fft'] = pd.Series(y_list_fft).apply(lambda x: np.median(x))
X_train['z_median_fft'] = pd.Series(z_list_fft).apply(lambda x: np.median(x))

# FFT median abs dev 
X_train['x_mad_fft'] = pd.Series(x_list_fft).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['y_mad_fft'] = pd.Series(y_list_fft).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['z_mad_fft'] = pd.Series(z_list_fft).apply(lambda x: np.median(np.absolute(x - np.median(x))))

# FFT Interquartile range
X_train['x_IQR_fft'] = pd.Series(x_list_fft).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['y_IQR_fft'] = pd.Series(y_list_fft).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['z_IQR_fft'] = pd.Series(z_list_fft).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))

# FFT values above mean
X_train['x_above_mean_fft'] = pd.Series(x_list_fft).apply(lambda x: np.sum(x > x.mean())) 
X_train['y_above_mean_fft'] = pd.Series(y_list_fft).apply(lambda x: np.sum(x > x.mean()))
X_train['z_above_mean_fft'] = pd.Series(z_list_fft).apply(lambda x: np.sum(x > x.mean()))

# FFT skewness
X_train['x_skewness_fft'] = pd.Series(x_list_fft).apply(lambda x: stats.skew(x))
X_train['y_skewness_fft'] = pd.Series(y_list_fft).apply(lambda x: stats.skew(x))
X_train['z_skewness_fft'] = pd.Series(z_list_fft).apply(lambda x: stats.skew(x))

# FFT kurtosis
X_train['x_kurtosis_fft'] = pd.Series(x_list_fft).apply(lambda x: stats.kurtosis(x))
X_train['y_kurtosis_fft'] = pd.Series(y_list_fft).apply(lambda x: stats.kurtosis(x))
X_train['z_kurtosis_fft'] = pd.Series(z_list_fft).apply(lambda x: stats.kurtosis(x))

# FFT energy
X_train['x_energy_fft'] = pd.Series(x_list_fft).apply(lambda x: np.sum(x**2)/50)
X_train['y_energy_fft'] = pd.Series(y_list_fft).apply(lambda x: np.sum(x**2)/50)
X_train['z_energy_fft'] = pd.Series(z_list_fft).apply(lambda x: np.sum(x**2/50))

# FFT avg resultant
X_train['avg_result_accl_fft'] = [i.mean() for i in ((pd.Series(x_list_fft)**2 + pd.Series(y_list_fft)**2 + pd.Series(z_list_fft)**2)**0.5)]

# FFT Signal magnitude area
X_train['sma_fft'] = pd.Series(x_list_fft).apply(lambda x: np.sum(abs(x)/50)) + pd.Series(y_list_fft).apply(lambda x: np.sum(abs(x)/50)) \
                     + pd.Series(z_list_fft).apply(lambda x: np.sum(abs(x)/50))

In [ ]:
X_train['x_argmax'] = pd.Series(x_list).apply(lambda x: np.argmax(x))
X_train['y_argmax'] = pd.Series(y_list).apply(lambda x: np.argmax(x)) 
X_train['z_argmax'] = pd.Series(z_list).apply(lambda x: np.argmax(x))

# index of min value in time domain
X_train['x_argmin'] = pd.Series(x_list).apply(lambda x: np.argmin(x))
X_train['y_argmin'] = pd.Series(y_list).apply(lambda x: np.argmin(x))
X_train['z_argmin'] = pd.Series(z_list).apply(lambda x: np.argmin(x))

# absolute difference between above indices
X_train['x_arg_diff'] = abs(X_train['x_argmax'] - X_train['x_argmin'])
X_train['y_arg_diff'] = abs(X_train['y_argmax'] - X_train['y_argmin'])
X_train['z_arg_diff'] = abs(X_train['z_argmax'] - X_train['z_argmin'])

# index of max value in frequency domain
X_train['x_argmax_fft'] = pd.Series(x_list_fft).apply(lambda x: np.argmax(np.abs(np.fft.fft(x))[1:51]))
X_train['y_argmax_fft'] = pd.Series(y_list_fft).apply(lambda x: np.argmax(np.abs(np.fft.fft(x))[1:51]))
X_train['z_argmax_fft'] = pd.Series(z_list_fft).apply(lambda x: np.argmax(np.abs(np.fft.fft(x))[1:51]))

# index of min value in frequency domain
X_train['x_argmin_fft'] = pd.Series(x_list_fft).apply(lambda x: np.argmin(np.abs(np.fft.fft(x))[1:51]))
X_train['y_argmin_fft'] = pd.Series(y_list_fft).apply(lambda x: np.argmin(np.abs(np.fft.fft(x))[1:51]))
X_train['z_argmin_fft'] = pd.Series(z_list_fft).apply(lambda x: np.argmin(np.abs(np.fft.fft(x))[1:51]))

# absolute difference between above indices
X_train['x_arg_diff_fft'] = abs(X_train['x_argmax_fft'] - X_train['x_argmin_fft'])
X_train['y_arg_diff_fft'] = abs(X_train['y_argmax_fft'] - X_train['y_argmin_fft'])
X_train['z_arg_diff_fft'] = abs(X_train['z_argmax_fft'] - X_train['z_argmin_fft'])

<ipython-input-10-a82af1227f2d>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train['z_argmax_fft'] = pd.Series(z_list_fft).apply(lambda x: np.argmax(np.abs(np.fft.fft(x))[1:51]))
<ipython-input-10-a82af1227f2d>:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train['x_argmin_fft'] = pd.Series(x_list_fft).apply(lambda x: np.argmin(np.abs(np.fft.fft(x))[1:51]))
<ipython-input-10-a82af1227f2d>:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which ha

In [ ]:
X_train.head()

,tac,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,...,z_arg_diff,x_argmax_fft,y_argmax_fft,z_argmax_fft,x_argmin_fft,y_argmin_fft,z_argmin_fft,x_arg_diff_fft,y_arg_diff_fft,z_arg_diff_fft
0,0,-0.000522,-0.001690,-0.001607,0.012240,0.007164,0.010232,0.008288,0.005044,0.007790,...,10,5,6,2,14,16,21,9,10,19
1,0,-0.004428,0.000969,-0.000996,0.009895,0.008558,0.012174,0.007525,0.006547,0.008964,...,70,9,2,2,28,7,24,19,5,22
2,0,-0.002430,0.003917,0.000383,0.018988,0.013274,0.018427,0.013756,0.009328,0.013305,...,14,1,2,1,22,19,19,21,17,18
3,0,-0.003570,0.003626,0.000294,0.026215,0.014554,0.020006,0.020313,0.010637,0.013874,...,8,3,2,18,24,1,23,21,1,5
4,0,-0.003775,0.003096,-0.000126,0.023270,0.012352,0.014846,0.018516,0.010172,0.010447,...,4,4,23,3,17,24,12,13,1,9


In [ ]:
X_train

,tac,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,...,z_arg_diff,x_argmax_fft,y_argmax_fft,z_argmax_fft,x_argmin_fft,y_argmin_fft,z_argmin_fft,x_arg_diff_fft,y_arg_diff_fft,z_arg_diff_fft
0,0,-0.000522,-0.001690,-0.001607,0.012240,0.007164,0.010232,0.008288,0.005044,0.007790,...,10,5,6,2,14,16,21,9,10,19
1,0,-0.004428,0.000969,-0.000996,0.009895,0.008558,0.012174,0.007525,0.006547,0.008964,...,70,9,2,2,28,7,24,19,5,22
2,0,-0.002430,0.003917,0.000383,0.018988,0.013274,0.018427,0.013756,0.009328,0.013305,...,14,1,2,1,22,19,19,21,17,18
3,0,-0.003570,0.003626,0.000294,0.026215,0.014554,0.020006,0.020313,0.010637,0.013874,...,8,3,2,18,24,1,23,21,1,5
4,0,-0.003775,0.003096,-0.000126,0.023270,0.012352,0.014846,0.018516,0.010172,0.010447,...,4,4,23,3,17,24,12,13,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6211,1,-0.015735,0.008933,-0.015479,0.102667,0.107518,0.080775,0.079306,0.081922,0.062237,...,2,24,1,0,8,23,24,16,22,24
6212,1,-0.016903,0.003963,-0.018763,0.112971,0.117893,0.115033,0.087053,0.093859,0.083551,...,17,6,1,7,22,9,33,16,8,26
6213,1,0.016968,-0.024163,-0.011454,0.134635,0.128519,0.145194,0.104591,0.098087,0.108781,...,73,12,10,9,14,24,4,2,14,5
6214,1,0.042983,0.020766,-0.037998,0.198713,0.178275,0.144190,0.155722,0.145211,0.109714,...,65,15,27,8,31,3,13,16,24,5


In [ ]:
for col in X_train.columns:
    print(col)

tac
x_mean
y_mean
z_mean
x_std
y_std
z_std
x_aad
y_aad
z_aad
x_min
y_min
z_min
x_max
y_max
z_max
x_maxmin_diff
y_maxmin_diff
z_maxmin_diff
x_median
y_median
z_median
x_mad
y_mad
z_mad
x_IQR
y_IQR
z_IQR
x_neg_count
y_neg_count
z_neg_count
x_pos_count
y_pos_count
z_pos_count
x_above_mean
y_above_mean
z_above_mean
x_skewness
y_skewness
z_skewness
x_kurtosis
y_kurtosis
z_kurtosis
x_energy
y_energy
z_energy
avg_result_accl
sma
x_mean_fft
y_mean_fft
z_mean_fft
x_std_fft
y_std_fft
z_std_fft
x_aad_fft
y_aad_fft
z_aad_fft
x_min_fft
y_min_fft
z_min_fft
x_max_fft
y_max_fft
z_max_fft
x_maxmin_diff_fft
y_maxmin_diff_fft
z_maxmin_diff_fft
x_median_fft
y_median_fft
z_median_fft
x_mad_fft
y_mad_fft
z_mad_fft
x_IQR_fft
y_IQR_fft
z_IQR_fft
x_above_mean_fft
y_above_mean_fft
z_above_mean_fft
x_skewness_fft
y_skewness_fft
z_skewness_fft
x_kurtosis_fft
y_kurtosis_fft
z_kurtosis_fft
x_energy_fft
y_energy_fft
z_energy_fft
avg_result_accl_fft
sma_fft
x_argmax
y_argmax
z_argmax
x_argmin
y_argmin
z_argmin
x_arg_

In [ ]:
# X_train.to_csv('DC6359_feature_engg_updated.csv')

In [ ]:
# from google.colab import files
# X_train.to_csv('DC6359_feature_engg_updated.csv') 
# files.download('DC6359_feature_engg_updated.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>